In [3]:
"""
AIRBNB PRICE PREDICTION - DATA CLEANING PIPELINE
================================================
This notebook implements a clean data preprocessing pipeline with:
1. Calendar aggregation with listings
2. Train/Val/Test split (60/20/20)
3. Preprocessing functions applied to each split
4. No data leakage
"""

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sys
import io

# Set UTF-8 encoding for Windows

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
import re

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("=" * 80)
print("✓ All libraries imported successfully")
print("=" * 80)

✓ All libraries imported successfully


In [4]:
"""
STEP 1: LOAD DATA
=================
Load listings and calendar datasets
"""
print("\n" + "=" * 80)
print("STEP 1: LOAD DATA")
print("=" * 80)

# Load listings
listings_df = pd.read_csv('main_dataset/listings_details.csv')
print(f"✓ Listings loaded: {listings_df.shape}")

# Load calendar
calendar_df = pd.read_csv('main_dataset/calendar.csv')
print(f"✓ Calendar loaded: {calendar_df.shape}")
print(f"  Columns: {calendar_df.columns.tolist()}")


STEP 1: LOAD DATA
✓ Listings loaded: (20030, 96)
✓ Calendar loaded: (7310950, 4)
  Columns: ['listing_id', 'date', 'available', 'price']


In [5]:
"""
STEP 2: CALENDAR AGGREGATION
=============================
Aggregate calendar data by listing_id to create new features
"""
print("\n" + "=" * 80)
print("STEP 2: CALENDAR AGGREGATION")
print("=" * 80)

# Clean calendar price column
def clean_calendar_price(price_str):
    if pd.isna(price_str):
        return np.nan
    if isinstance(price_str, (int, float)):
        return float(price_str)
    return float(str(price_str).replace('$', '').replace(',', ''))

calendar_df['price_clean'] = calendar_df['price'].apply(clean_calendar_price)

# Convert available column
calendar_df['is_available'] = calendar_df['available'].map({'t': 1, 'f': 0})

# Convert date to datetime
calendar_df['date'] = pd.to_datetime(calendar_df['date'], errors='coerce')

print(f"✓ Calendar data cleaned")
print(f"  Available values: {calendar_df['is_available'].value_counts().to_dict()}")

# Aggregate calendar features by listing_id
calendar_agg = (
    calendar_df
    .groupby('listing_id')
    .agg(
        avg_calendar_price=('price_clean', 'mean'),
        min_calendar_price=('price_clean', 'min'),
        max_calendar_price=('price_clean', 'max'),
        availability_rate=('is_available', 'mean'),
        calendar_days_count=('date', 'count'),
        calendar_available_days=('is_available', 'sum')
    )
    .reset_index()
)

print(f"✓ Calendar aggregated: {calendar_agg.shape}")
print(f"  Features created: {calendar_agg.columns.tolist()}")

# Merge with listings
df = listings_df.merge(
    calendar_agg,
    left_on='id',
    right_on='listing_id',
    how='left'
)

# Drop duplicate listing_id column from merge
if 'listing_id' in df.columns:
    df = df.drop(columns=['listing_id'])

print(f"✓ Merged with listings: {df.shape}")
print(f"  Missing calendar data: {df['avg_calendar_price'].isnull().sum()} listings")


STEP 2: CALENDAR AGGREGATION
✓ Calendar data cleaned
  Available values: {0: 6110879, 1: 1200071}
✓ Calendar aggregated: (20030, 7)
  Features created: ['listing_id', 'avg_calendar_price', 'min_calendar_price', 'max_calendar_price', 'availability_rate', 'calendar_days_count', 'calendar_available_days']
✓ Merged with listings: (20030, 102)
  Missing calendar data: 9344 listings


In [7]:
"""
STEP 3: TRAIN/VAL/TEST SPLIT (60/20/20)
========================================
Split BEFORE any transformations to prevent data leakage
"""
print("\n" + "=" * 80)
print("STEP 3: TRAIN/VAL/TEST SPLIT (60/20/20)")
print("=" * 80)

# Remove duplicates first
df = df.drop_duplicates(subset=['id'], keep='first')
print(f"✓ Removed duplicates. Shape: {df.shape}")

# Convert price to numeric FIRST (it's loaded as string with $ signs)
if 'price' in df.columns:
    def clean_price(price_str):
        if pd.isna(price_str):
            return np.nan
        if isinstance(price_str, (int, float)):
            return float(price_str)
        # Remove $ and commas, then convert to float
        return float(str(price_str).replace('$', '').replace(',', ''))
    
    df['price'] = df['price'].apply(clean_price)
    print(f"✓ Converted price column to numeric")

# Remove rows with missing target variable (price)
if 'price' in df.columns:
    before_count = len(df)
    df = df[df['price'].notna()]
    removed = before_count - len(df)
    if removed > 0:
        print(f"✓ Removed {removed} rows with missing price")

# Remove zero or negative prices
if 'price' in df.columns:
    before_count = len(df)
    df = df[df['price'] > 0]
    removed = before_count - len(df)
    if removed > 0:
        print(f"✓ Removed {removed} rows with zero/negative price")

print(f"Final shape before split: {df.shape}")

# Separate features and target
X = df.drop(columns=['price'])
y = df['price']

# Split: 60% train, 20% val, 20% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, shuffle=True
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True
)

print(f"\n✓ Split completed:")
print(f"  Train: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Val:   {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test:  {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"  Features: {X_train.shape[1]}")

# Store train indices for reference
train_ids = X_train['id'].values if 'id' in X_train.columns else None
print(f"\n✓ Data split complete. Ready for preprocessing functions.")


STEP 3: TRAIN/VAL/TEST SPLIT (60/20/20)
✓ Removed duplicates. Shape: (20030, 102)
✓ Converted price column to numeric
✓ Removed 2 rows with zero/negative price
Final shape before split: (20028, 102)

✓ Split completed:
  Train: 12,016 samples (60.0%)
  Val:   4,006 samples (20.0%)
  Test:  4,006 samples (20.0%)
  Features: 101

✓ Data split complete. Ready for preprocessing functions.


In [8]:
"""
STEP 4: DEFINE PREPROCESSING FUNCTIONS
=======================================
All preprocessing functions that will be applied to split data
"""
print("\n" + "=" * 80)
print("STEP 4: DEFINE PREPROCESSING FUNCTIONS")
print("=" * 80)

def preprocess_data(X_train, X_val, X_test, y_train, verbose=True):
    """
    Complete preprocessing pipeline that fits on train and transforms all splits.
    
    Parameters:
    -----------
    X_train, X_val, X_test : pandas DataFrames
        Feature matrices for each split
    y_train : pandas Series
        Target variable for training set (needed for target encoding)
    verbose : bool
        Whether to print progress
        
    Returns:
    --------
    tuple : (X_train_processed, X_val_processed, X_test_processed, feature_names, encoders)
    """
    
    # Make copies to avoid modifying originals
    X_train = X_train.copy()
    X_val = X_val.copy()
    X_test = X_test.copy()
    
    encoders = {}  # Store fitted encoders/transformers
    
    if verbose:
        print("\n" + "-" * 80)
        print("1. DROPPING IRRELEVANT COLUMNS")
        print("-" * 80)
    
    # Drop irrelevant columns
    cols_to_drop = [
        'listing_url', 'scrape_id', 'last_scraped', 'thumbnail_url', 'medium_url',
        'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_thumbnail_url',
        'host_picture_url', 'license', 'jurisdiction_names', 'calendar_last_scraped',
        'experiences_offered', 'neighbourhood_group_cleansed', 'id'
    ]
    
    # Drop columns with 100% missing in TRAINING data only
    missing_100_cols = X_train.columns[X_train.isnull().mean() == 1.0].tolist()
    cols_to_drop.extend(missing_100_cols)
    
    cols_dropped = [c for c in cols_to_drop if c in X_train.columns]
    X_train = X_train.drop(columns=cols_dropped, errors='ignore')
    X_val = X_val.drop(columns=cols_dropped, errors='ignore')
    X_test = X_test.drop(columns=cols_dropped, errors='ignore')
    
    if verbose:
        print(f"  ✓ Dropped {len(cols_dropped)} columns")
        print(f"  Shape: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")
    
    # ========================================================================
    # 2. TYPE CONVERSION
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("2. TYPE CONVERSION")
        print("-" * 80)
    
    def clean_price(price_str):
        if pd.isna(price_str):
            return np.nan
        if isinstance(price_str, (int, float)):
            return float(price_str)
        return float(str(price_str).replace('$', '').replace(',', ''))
    
    # Price columns
    price_cols = ['weekly_price', 'monthly_price', 'security_deposit',
                  'cleaning_fee', 'extra_people']
    for col in price_cols:
        if col in X_train.columns:
            X_train[col] = X_train[col].apply(clean_price)
            X_val[col] = X_val[col].apply(clean_price)
            X_test[col] = X_test[col].apply(clean_price)
    
    # Percentage columns
    percentage_cols = ['host_response_rate', 'host_acceptance_rate']
    for col in percentage_cols:
        if col in X_train.columns:
            X_train[col] = X_train[col].apply(lambda x: float(str(x).replace('%', '')) / 100 if pd.notna(x) else np.nan)
            X_val[col] = X_val[col].apply(lambda x: float(str(x).replace('%', '')) / 100 if pd.notna(x) else np.nan)
            X_test[col] = X_test[col].apply(lambda x: float(str(x).replace('%', '')) / 100 if pd.notna(x) else np.nan)
    
    # Boolean columns
    bool_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
                 'is_location_exact', 'has_availability', 'instant_bookable',
                 'is_business_travel_ready', 'require_guest_profile_picture',
                 'require_guest_phone_verification', 'requires_license']
    for col in bool_cols:
        if col in X_train.columns:
            X_train[col] = X_train[col].map({'t': True, 'f': False, True: True, False: False})
            X_val[col] = X_val[col].map({'t': True, 'f': False, True: True, False: False})
            X_test[col] = X_test[col].map({'t': True, 'f': False, True: True, False: False})
    
    # Numeric columns
    numeric_cols = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'guests_included',
                    'minimum_nights', 'maximum_nights', 'availability_30', 'availability_60',
                    'availability_90', 'availability_365', 'number_of_reviews',
                    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                    'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                    'review_scores_value', 'calculated_host_listings_count', 'reviews_per_month',
                    'host_listings_count', 'host_total_listings_count', 'square_feet',
                    'latitude', 'longitude']
    for col in numeric_cols:
        if col in X_train.columns:
            X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
            X_val[col] = pd.to_numeric(X_val[col], errors='coerce')
            X_test[col] = pd.to_numeric(X_test[col], errors='coerce')
    
    # Date columns
    date_cols = ['host_since', 'first_review', 'last_review']
    for col in date_cols:
        if col in X_train.columns:
            X_train[col] = pd.to_datetime(X_train[col], errors='coerce')
            X_val[col] = pd.to_datetime(X_val[col], errors='coerce')
            X_test[col] = pd.to_datetime(X_test[col], errors='coerce')
    
    if verbose:
        print("  ✓ Type conversion completed")
    
    # ========================================================================
    # 3. LOGIC ERROR FIXING
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("3. LOGIC ERROR FIXING")
        print("-" * 80)
    
    # Fix min > max nights (set to NaN, will be imputed later)
    if 'minimum_nights' in X_train.columns and 'maximum_nights' in X_train.columns:
        mask_train = X_train['minimum_nights'] > X_train['maximum_nights']
        mask_val = X_val['minimum_nights'] > X_val['maximum_nights']
        mask_test = X_test['minimum_nights'] > X_test['maximum_nights']
        
        X_train.loc[mask_train, ['minimum_nights', 'maximum_nights']] = np.nan
        X_val.loc[mask_val, ['minimum_nights', 'maximum_nights']] = np.nan
        X_test.loc[mask_test, ['minimum_nights', 'maximum_nights']] = np.nan
        
        if verbose and mask_train.sum() > 0:
            print(f"  ✓ Fixed min/max night errors: Train={mask_train.sum()}, Val={mask_val.sum()}, Test={mask_test.sum()}")
    
    # ========================================================================
    # 4. DROP HIGH MISSING COLUMNS (Based on TRAINING data)
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("4. DROP HIGH MISSING COLUMNS")
        print("-" * 80)
    
    missing_pct = (X_train.isnull().sum() / len(X_train) * 100)
    cols_to_drop_missing = missing_pct[missing_pct > 70].index.tolist()
    
    if cols_to_drop_missing:
        X_train = X_train.drop(columns=cols_to_drop_missing)
        X_val = X_val.drop(columns=cols_to_drop_missing)
        X_test = X_test.drop(columns=cols_to_drop_missing)
        if verbose:
            print(f"  ✓ Dropped {len(cols_to_drop_missing)} columns with >70% missing in training data")
    
    # ========================================================================
    # 5. DOMAIN KNOWLEDGE FILLS (Safe before other imputation)
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("5. DOMAIN KNOWLEDGE FILLS")
        print("-" * 80)
    
    # Security deposit: 0 means no deposit
    if 'security_deposit' in X_train.columns:
        X_train['security_deposit'] = X_train['security_deposit'].fillna(0)
        X_val['security_deposit'] = X_val['security_deposit'].fillna(0)
        X_test['security_deposit'] = X_test['security_deposit'].fillna(0)
        if verbose:
            print("  ✓ Filled security_deposit with 0")
    
    # Host neighbourhood from listing neighbourhood
    if 'host_neighbourhood' in X_train.columns and 'neighbourhood_cleansed' in X_train.columns:
        X_train['host_neighbourhood'] = X_train['host_neighbourhood'].fillna(X_train['neighbourhood_cleansed'])
        X_val['host_neighbourhood'] = X_val['host_neighbourhood'].fillna(X_val['neighbourhood_cleansed'])
        X_test['host_neighbourhood'] = X_test['host_neighbourhood'].fillna(X_test['neighbourhood_cleansed'])
        if verbose:
            print("  ✓ Filled host_neighbourhood from neighbourhood_cleansed")
    
    # Review scores: 0 means no reviews
    review_cols = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                   'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                   'review_scores_value', 'reviews_per_month']
    for col in review_cols:
        if col in X_train.columns:
            X_train[col] = X_train[col].fillna(0)
            X_val[col] = X_val[col].fillna(0)
            X_test[col] = X_test[col].fillna(0)
    if verbose:
        print(f"  ✓ Filled {len([c for c in review_cols if c in X_train.columns])} review columns with 0")
    
    # Text columns: Unknown
    text_cols = ['notes', 'transit', 'access', 'interaction', 'house_rules',
                 'neighborhood_overview', 'host_about', 'host_response_time']
    for col in text_cols:
        if col in X_train.columns:
            X_train[col] = X_train[col].fillna('Unknown')
            X_val[col] = X_val[col].fillna('Unknown')
            X_test[col] = X_test[col].fillna('Unknown')
    
    # Categorical columns: Use TRAINING mode
    cat_cols = ['name', 'summary', 'space', 'description', 'host_name', 'host_location',
                'neighbourhood', 'city', 'state', 'zipcode', 'market']
    for col in cat_cols:
        if col in X_train.columns and X_train[col].isnull().sum() > 0:
            mode_val = X_train[col].mode()[0] if len(X_train[col].mode()) > 0 else 'Unknown'
            X_train[col] = X_train[col].fillna(mode_val)
            X_val[col] = X_val[col].fillna(mode_val)
            X_test[col] = X_test[col].fillna(mode_val)
            encoders[f'{col}_mode'] = mode_val
    
    # Boolean: False
    for col in X_train.select_dtypes(include=['bool']).columns:
        if X_train[col].isnull().sum() > 0:
            X_train[col] = X_train[col].fillna(False)
            X_val[col] = X_val[col].fillna(False)
            X_test[col] = X_test[col].fillna(False)
    
    if verbose:
        print("  ✓ Domain knowledge fills completed")
    
    # ========================================================================
    # 6. FEATURE ENGINEERING
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("6. FEATURE ENGINEERING")
        print("-" * 80)
    
    reference_date = pd.Timestamp('2018-12-06')
    
    # Date features - Process each dataset directly (FIXED: proper variable reference)
    for df in [X_train, X_val, X_test]:
        if 'host_since' in df.columns:
            df['host_tenure_days'] = (reference_date - df['host_since']).dt.days
            df['host_tenure_years'] = df['host_tenure_days'] / 365.25
            df['host_since_year'] = df['host_since'].dt.year
            df['host_since_month'] = df['host_since'].dt.month
            df['host_since_dayofweek'] = df['host_since'].dt.dayofweek
            df['host_since_month_sin'] = np.sin(2 * np.pi * df['host_since_month'] / 12)
            df['host_since_month_cos'] = np.cos(2 * np.pi * df['host_since_month'] / 12)
        
        if 'first_review' in df.columns:
            df['days_since_first_review'] = (reference_date - df['first_review']).dt.days
            df['days_since_first_review'] = df['days_since_first_review'].fillna(0)
        
        if 'last_review' in df.columns:
            df['days_since_last_review'] = (reference_date - df['last_review']).dt.days
            df['days_since_last_review'] = df['days_since_last_review'].fillna(9999)
        
        if 'first_review' in df.columns and 'last_review' in df.columns:
            df['review_period_days'] = (df['last_review'] - df['first_review']).dt.days
            df['review_period_days'] = df['review_period_days'].fillna(0)
        
        # Text features
        text_columns = ['name', 'summary', 'space', 'description', 'neighborhood_overview',
                        'notes', 'transit', 'access', 'interaction', 'house_rules']
        for col in text_columns:
            if col in df.columns:
                df[f'{col}_length'] = df[col].astype(str).str.len()
                df[f'{col}_word_count'] = df[col].astype(str).str.split().str.len()
        
        # Amenities features
        if 'amenities' in df.columns:
            df['amenities_count'] = df['amenities'].astype(str).str.count(',') + 1
            df['amenities_count'] = df['amenities_count'].replace({1: 0})
            df['has_wifi'] = df['amenities'].str.contains('wifi|internet', case=False, na=False).astype(int)
            df['has_kitchen'] = df['amenities'].str.contains('kitchen', case=False, na=False).astype(int)
            df['has_tv'] = df['amenities'].str.contains('tv', case=False, na=False).astype(int)
            df['has_parking'] = df['amenities'].str.contains('parking', case=False, na=False).astype(int)
            df['has_ac'] = df['amenities'].str.contains('air conditioning|ac', case=False, na=False).astype(int)
            df['has_heating'] = df['amenities'].str.contains('heating', case=False, na=False).astype(int)
        
        # Host verifications
        if 'host_verifications' in df.columns:
            df['host_verifications_count'] = df['host_verifications'].astype(str).str.count(',') + 1
    
    if verbose:
        print("  ✓ Feature engineering completed")
    
    # ========================================================================
    # 7. CLEANUP - DROP ORIGINAL TEXT/DATE COLUMNS
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("7. CLEANUP")
        print("-" * 80)
    
    # Drop original date columns
    date_cols_to_drop = ['host_since', 'first_review', 'last_review']
    X_train = X_train.drop(columns=[c for c in date_cols_to_drop if c in X_train.columns])
    X_val = X_val.drop(columns=[c for c in date_cols_to_drop if c in X_val.columns])
    X_test = X_test.drop(columns=[c for c in date_cols_to_drop if c in X_test.columns])
    
    # Drop original text columns
    text_cols_to_drop = ['name', 'summary', 'space', 'description', 'neighborhood_overview',
                         'notes', 'transit', 'access', 'interaction', 'house_rules',
                         'amenities', 'host_verifications', 'host_about']
    X_train = X_train.drop(columns=[c for c in text_cols_to_drop if c in X_train.columns], errors='ignore')
    X_val = X_val.drop(columns=[c for c in text_cols_to_drop if c in X_val.columns], errors='ignore')
    X_test = X_test.drop(columns=[c for c in text_cols_to_drop if c in X_test.columns], errors='ignore')
    
    # Drop other irrelevant columns
    other_drops = ['street', 'city', 'state', 'zipcode', 'market', 'smart_location',
                   'country', 'country_code', 'calendar_updated']
    X_train = X_train.drop(columns=[c for c in other_drops if c in X_train.columns], errors='ignore')
    X_val = X_val.drop(columns=[c for c in other_drops if c in X_val.columns], errors='ignore')
    X_test = X_test.drop(columns=[c for c in other_drops if c in X_test.columns], errors='ignore')
    
    if verbose:
        print(f"  ✓ Dropped original text/date columns")
        print(f"  Shape: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")
    
    # ========================================================================
    # 8. CATEGORICAL ENCODING
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("8. CATEGORICAL ENCODING")
        print("-" * 80)
    
    # Identify categorical features
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    
    # Separate by cardinality
    low_cardinality = []
    target_encode_cols = []
    
    for col in categorical_features:
        n_unique = X_train[col].nunique()
        if n_unique < 10:
            low_cardinality.append(col)
        else:
            target_encode_cols.append(col)
    
    if verbose:
        print(f"  Low cardinality (<10 unique): {len(low_cardinality)} features")
        print(f"  High cardinality (>=10 unique): {len(target_encode_cols)} features (will target encode)")
    
    # One-hot encode low cardinality
    if low_cardinality:
        X_train = pd.get_dummies(X_train, columns=low_cardinality, prefix=low_cardinality,
                                drop_first=True, dtype=int)
        X_val = pd.get_dummies(X_val, columns=low_cardinality, prefix=low_cardinality,
                              drop_first=True, dtype=int)
        X_test = pd.get_dummies(X_test, columns=low_cardinality, prefix=low_cardinality,
                               drop_first=True, dtype=int)
        
        # Align columns
        train_cols = set(X_train.columns)
        val_cols = set(X_val.columns)
        test_cols = set(X_test.columns)
        all_cols = train_cols.union(val_cols).union(test_cols)
        
        for col in all_cols:
            if col not in X_train.columns:
                X_train[col] = 0
            if col not in X_val.columns:
                X_val[col] = 0
            if col not in X_test.columns:
                X_test[col] = 0
        
        # Ensure same column order
        X_val = X_val[X_train.columns]
        X_test = X_test[X_train.columns]
        
        if verbose:
            print(f"  ✓ One-hot encoded {len(low_cardinality)} features")
    
    # Target encode high cardinality (FIT ON TRAIN ONLY!)
    if target_encode_cols:
        if verbose:
            print(f"  Target encoding {len(target_encode_cols)} high-cardinality features...")
        
        for col in target_encode_cols:
            if col in X_train.columns:
                # Calculate means from TRAINING DATA ONLY
                train_with_target = X_train[[col]].copy()
                train_with_target['price'] = y_train.values
                target_means = train_with_target.groupby(col)['price'].mean()
                global_mean = y_train.mean()
                
                # Store encoder
                encoders[f'{col}_target_encoder'] = {'means': target_means, 'global_mean': global_mean}
                
                # Apply to all sets
                X_train[f'{col}_target_encoded'] = X_train[col].map(target_means).fillna(global_mean)
                X_val[f'{col}_target_encoded'] = X_val[col].map(target_means).fillna(global_mean)
                X_test[f'{col}_target_encoded'] = X_test[col].map(target_means).fillna(global_mean)
        
        # Drop original columns
        X_train = X_train.drop(columns=target_encode_cols)
        X_val = X_val.drop(columns=target_encode_cols)
        X_test = X_test.drop(columns=target_encode_cols)
        
        if verbose:
            print(f"  ✓ Target encoding completed")
    
    # Convert booleans to int
    for col in X_train.select_dtypes(include=['bool']).columns:
        X_train[col] = X_train[col].astype(int)
        X_val[col] = X_val[col].astype(int)
        X_test[col] = X_test[col].astype(int)
    
    if verbose:
        print(f"  Final shape after encoding: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")
    
    # ========================================================================
    # 9. HANDLE REMAINING MISSING VALUES (Based on TRAINING data)
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("9. HANDLE REMAINING MISSING VALUES")
        print("-" * 80)
        print(f"  NaN count BEFORE: Train={X_train.isnull().sum().sum()}, " +
              f"Val={X_val.isnull().sum().sum()}, Test={X_test.isnull().sum().sum()}")
    
    if X_train.isnull().sum().sum() > 0:
        # Impute with TRAINING median
        numeric_cols = X_train.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if X_train[col].isnull().sum() > 0:
                train_median = X_train[col].median()
                encoders[f'{col}_median'] = train_median
                
                X_train[col] = X_train[col].fillna(train_median)
                X_val[col] = X_val[col].fillna(train_median)
                X_test[col] = X_test[col].fillna(train_median)
    
    # Final safety check
    if X_train.isnull().sum().sum() > 0 or X_val.isnull().sum().sum() > 0 or X_test.isnull().sum().sum() > 0:
        X_train = X_train.fillna(0)
        X_val = X_val.fillna(0)
        X_test = X_test.fillna(0)
        if verbose:
            print("  ✓ Filled remaining NaN with 0")
    
    if verbose:
        print(f"  NaN count AFTER: Train={X_train.isnull().sum().sum()}, " +
              f"Val={X_val.isnull().sum().sum()}, Test={X_test.isnull().sum().sum()}")
    
    # ========================================================================
    # 10. OUTLIER TREATMENT (Based on TRAINING data)
    # ========================================================================
    if verbose:
        print("\n" + "-" * 80)
        print("10. OUTLIER TREATMENT")
        print("-" * 80)
    
    # Domain-based caps
    if 'minimum_nights' in X_train.columns:
        X_train['minimum_nights'] = X_train['minimum_nights'].clip(upper=365)
        X_val['minimum_nights'] = X_val['minimum_nights'].clip(upper=365)
        X_test['minimum_nights'] = X_test['minimum_nights'].clip(upper=365)
        if verbose:
            print("  ✓ Capped minimum_nights at 365")
    
    if 'maximum_nights' in X_train.columns:
        X_train['maximum_nights'] = X_train['maximum_nights'].clip(upper=730)
        X_val['maximum_nights'] = X_val['maximum_nights'].clip(upper=730)
        X_test['maximum_nights'] = X_test['maximum_nights'].clip(upper=730)
        if verbose:
            print("  ✓ Capped maximum_nights at 730")
    
    if 'accommodates' in X_train.columns:
        X_train['accommodates'] = X_train['accommodates'].clip(upper=16)
        X_val['accommodates'] = X_val['accommodates'].clip(upper=16)
        X_test['accommodates'] = X_test['accommodates'].clip(upper=16)
        if verbose:
            print("  ✓ Capped accommodates at 16")
    
    # Winsorize based on TRAINING quantiles
    winsorize_cols = ['cleaning_fee', 'security_deposit']
    for col in winsorize_cols:
        if col in X_train.columns:
            cap_val = X_train[col].quantile(0.99)
            encoders[f'{col}_99th'] = cap_val
            
            X_train[col] = X_train[col].clip(upper=cap_val)
            X_val[col] = X_val[col].clip(upper=cap_val)
            X_test[col] = X_test[col].clip(upper=cap_val)
            
            if verbose:
                print(f"  ✓ Winsorized {col} at 99th percentile: {cap_val:.2f}")
    
    if verbose:
        print("\n" + "-" * 80)
        print("✅ PREPROCESSING COMPLETE!")
        print("-" * 80)
        print(f"  Final shapes:")
        print(f"    Train: {X_train.shape}")
        print(f"    Val:   {X_val.shape}")
        print(f"    Test:  {X_test.shape}")
    
    feature_names = X_train.columns.tolist()
    
    return X_train, X_val, X_test, feature_names, encoders


print("✓ Preprocessing function defined successfully")



STEP 4: DEFINE PREPROCESSING FUNCTIONS
✓ Preprocessing function defined successfully


In [9]:
"""
STEP 5: APPLY PREPROCESSING TO SPLIT DATA
==========================================
Apply the preprocessing function to train, val, and test sets
"""
print("\n" + "=" * 80)
print("STEP 5: APPLY PREPROCESSING TO SPLIT DATA")
print("=" * 80)

# Apply preprocessing
X_train_clean, X_val_clean, X_test_clean, feature_names, encoders = preprocess_data(
    X_train, X_val, X_test, y_train, verbose=True
)

print("\n" + "=" * 80)
print("✅ PREPROCESSING APPLIED SUCCESSFULLY!")
print("=" * 80)
print(f"\nFinal Dataset Summary:")
print(f"  Train:      {X_train_clean.shape[0]:,} samples × {X_train_clean.shape[1]} features")
print(f"  Validation: {X_val_clean.shape[0]:,} samples × {X_val_clean.shape[1]} features")
print(f"  Test:       {X_test_clean.shape[0]:,} samples × {X_test_clean.shape[1]} features")
print(f"\n  Total features: {len(feature_names)}")
print(f"  Encoders/transformers stored: {len(encoders)}")

# Display sample of calendar features
calendar_features = [col for col in feature_names if 'calendar' in col.lower() or 'availability' in col.lower()]
if calendar_features:
    print(f"\n📅 Calendar-derived features ({len(calendar_features)}):")
    for feat in calendar_features:
        print(f"  - {feat}")



STEP 5: APPLY PREPROCESSING TO SPLIT DATA

--------------------------------------------------------------------------------
1. DROPPING IRRELEVANT COLUMNS
--------------------------------------------------------------------------------
  ✓ Dropped 22 columns
  Shape: Train (12016, 83), Val (4006, 83), Test (4006, 83)

--------------------------------------------------------------------------------
2. TYPE CONVERSION
--------------------------------------------------------------------------------
  ✓ Type conversion completed

--------------------------------------------------------------------------------
3. LOGIC ERROR FIXING
--------------------------------------------------------------------------------

--------------------------------------------------------------------------------
4. DROP HIGH MISSING COLUMNS
--------------------------------------------------------------------------------
  ✓ Dropped 3 columns with >70% missing in training data

---------------------------------

In [10]:
"""
STEP 6: SCALING (FIT ON TRAIN, TRANSFORM ALL)
==============================================
Apply different scalers for model flexibility
"""
print("\n" + "=" * 80)
print("STEP 6: SCALING (FIT ON TRAIN, TRANSFORM ALL)")
print("=" * 80)

# StandardScaler (best for most algorithms)
scaler_standard = StandardScaler()
X_train_standard = scaler_standard.fit_transform(X_train_clean)
X_val_standard = scaler_standard.transform(X_val_clean)
X_test_standard = scaler_standard.transform(X_test_clean)

print(f"\n✓ StandardScaler applied:")
print(f"  Train - mean: {X_train_standard.mean():.6f}, std: {X_train_standard.std():.6f}")
print(f"  Val   - mean: {X_val_standard.mean():.6f}, std: {X_val_standard.std():.6f}")
print(f"  Test  - mean: {X_test_standard.mean():.6f}, std: {X_test_standard.std():.6f}")

# MinMaxScaler (for neural networks)
scaler_minmax = MinMaxScaler()
X_train_minmax = scaler_minmax.fit_transform(X_train_clean)
X_val_minmax = scaler_minmax.transform(X_val_clean)
X_test_minmax = scaler_minmax.transform(X_test_clean)

print(f"\n✓ MinMaxScaler applied:")
print(f"  Train - range: [{X_train_minmax.min():.6f}, {X_train_minmax.max():.6f}]")
print(f"  Val   - range: [{X_val_minmax.min():.6f}, {X_val_minmax.max():.6f}]")
print(f"  Test  - range: [{X_test_minmax.min():.6f}, {X_test_minmax.max():.6f}]")

# RobustScaler (for data with outliers)
scaler_robust = RobustScaler()
X_train_robust = scaler_robust.fit_transform(X_train_clean)
X_val_robust = scaler_robust.transform(X_val_clean)
X_test_robust = scaler_robust.transform(X_test_clean)

print(f"\n✓ RobustScaler applied:")
print(f"  Train - median: {np.median(X_train_robust):.6f}")
print(f"  Val   - median: {np.median(X_val_robust):.6f}")
print(f"  Test  - median: {np.median(X_test_robust):.6f}")

# Verify no NaN values after scaling
print(f"\n✅ NaN Check After Scaling:")
print(f"  StandardScaler: Train={np.isnan(X_train_standard).sum()}, Val={np.isnan(X_val_standard).sum()}, Test={np.isnan(X_test_standard).sum()}")
print(f"  MinMaxScaler:   Train={np.isnan(X_train_minmax).sum()}, Val={np.isnan(X_val_minmax).sum()}, Test={np.isnan(X_test_minmax).sum()}")
print(f"  RobustScaler:   Train={np.isnan(X_train_robust).sum()}, Val={np.isnan(X_val_robust).sum()}, Test={np.isnan(X_test_robust).sum()}")



STEP 6: SCALING (FIT ON TRAIN, TRANSFORM ALL)

✓ StandardScaler applied:
  Train - mean: 0.000000, std: 0.980196
  Val   - mean: -0.002531, std: 0.967533
  Test  - mean: 0.001433, std: 0.976170

✓ MinMaxScaler applied:
  Train - range: [0.000000, 1.000000]
  Val   - range: [-0.014484, 1.005000]
  Test  - range: [-0.037885, 1.105280]

✓ RobustScaler applied:
  Train - median: 0.000000
  Val   - median: 0.000000
  Test  - median: 0.000000

✅ NaN Check After Scaling:
  StandardScaler: Train=0, Val=0, Test=0
  MinMaxScaler:   Train=0, Val=0, Test=0
  RobustScaler:   Train=0, Val=0, Test=0


In [11]:
"""
STEP 7: SAVE PROCESSED DATA
============================
Save all processed datasets for modeling
"""
print("\n" + "=" * 80)
print("STEP 7: SAVE PROCESSED DATA")
print("=" * 80)

import os

# Create output directory
output_dir = 'processed_data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✓ Created directory: {output_dir}/")

# Save unscaled data (DataFrames) - FIXED: Reset index for both X and y
train_unscaled = pd.concat([X_train_clean.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
val_unscaled = pd.concat([X_val_clean.reset_index(drop=True), y_val.reset_index(drop=True)], axis=1)
test_unscaled = pd.concat([X_test_clean.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)

train_unscaled.to_csv(f'{output_dir}/train_unscaled.csv', index=False)
val_unscaled.to_csv(f'{output_dir}/val_unscaled.csv', index=False)
test_unscaled.to_csv(f'{output_dir}/test_unscaled.csv', index=False)

print(f"\n✓ Saved unscaled data (CSV):")
print(f"  - {output_dir}/train_unscaled.csv ({train_unscaled.shape})")
print(f"  - {output_dir}/val_unscaled.csv ({val_unscaled.shape})")
print(f"  - {output_dir}/test_unscaled.csv ({test_unscaled.shape})")

# Save scaled data (NumPy arrays) - StandardScaler
np.save(f'{output_dir}/X_train_standard.npy', X_train_standard)
np.save(f'{output_dir}/X_val_standard.npy', X_val_standard)
np.save(f'{output_dir}/X_test_standard.npy', X_test_standard)
np.save(f'{output_dir}/y_train.npy', y_train.values)
np.save(f'{output_dir}/y_val.npy', y_val.values)
np.save(f'{output_dir}/y_test.npy', y_test.values)

print(f"\n✓ Saved scaled data - StandardScaler (NumPy):")
print(f"  - {output_dir}/X_train_standard.npy")
print(f"  - {output_dir}/X_val_standard.npy")
print(f"  - {output_dir}/X_test_standard.npy")
print(f"  - {output_dir}/y_train.npy, y_val.npy, y_test.npy")

# Save scaled data (NumPy arrays) - MinMaxScaler
np.save(f'{output_dir}/X_train_minmax.npy', X_train_minmax)
np.save(f'{output_dir}/X_val_minmax.npy', X_val_minmax)
np.save(f'{output_dir}/X_test_minmax.npy', X_test_minmax)

print(f"\n✓ Saved scaled data - MinMaxScaler (NumPy):")
print(f"  - {output_dir}/X_train_minmax.npy")
print(f"  - {output_dir}/X_val_minmax.npy")
print(f"  - {output_dir}/X_test_minmax.npy")

# Save scaled data (NumPy arrays) - RobustScaler
np.save(f'{output_dir}/X_train_robust.npy', X_train_robust)
np.save(f'{output_dir}/X_val_robust.npy', X_val_robust)
np.save(f'{output_dir}/X_test_robust.npy', X_test_robust)

print(f"\n✓ Saved scaled data - RobustScaler (NumPy):")
print(f"  - {output_dir}/X_train_robust.npy")
print(f"  - {output_dir}/X_val_robust.npy")
print(f"  - {output_dir}/X_test_robust.npy")

# Save feature names
feature_names_df = pd.DataFrame({'feature': feature_names})
feature_names_df.to_csv(f'{output_dir}/feature_names.csv', index=False)
print(f"\n✓ Saved feature names:")
print(f"  - {output_dir}/feature_names.csv ({len(feature_names)} features)")

# Save scalers using pickle
import pickle

with open(f'{output_dir}/scaler_standard.pkl', 'wb') as f:
    pickle.dump(scaler_standard, f)

with open(f'{output_dir}/scaler_minmax.pkl', 'wb') as f:
    pickle.dump(scaler_minmax, f)

with open(f'{output_dir}/scaler_robust.pkl', 'wb') as f:
    pickle.dump(scaler_robust, f)

with open(f'{output_dir}/encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

print(f"\n✓ Saved scalers and encoders (pickle):")
print(f"  - {output_dir}/scaler_standard.pkl")
print(f"  - {output_dir}/scaler_minmax.pkl")
print(f"  - {output_dir}/scaler_robust.pkl")
print(f"  - {output_dir}/encoders.pkl")

print("\n" + "=" * 80)
print("✅ ALL DATA SAVED SUCCESSFULLY!")
print("=" * 80)



STEP 7: SAVE PROCESSED DATA
✓ Created directory: processed_data/

✓ Saved unscaled data (CSV):
  - processed_data/train_unscaled.csv ((12016, 103))
  - processed_data/val_unscaled.csv ((4006, 103))
  - processed_data/test_unscaled.csv ((4006, 103))

✓ Saved scaled data - StandardScaler (NumPy):
  - processed_data/X_train_standard.npy
  - processed_data/X_val_standard.npy
  - processed_data/X_test_standard.npy
  - processed_data/y_train.npy, y_val.npy, y_test.npy

✓ Saved scaled data - MinMaxScaler (NumPy):
  - processed_data/X_train_minmax.npy
  - processed_data/X_val_minmax.npy
  - processed_data/X_test_minmax.npy

✓ Saved scaled data - RobustScaler (NumPy):
  - processed_data/X_train_robust.npy
  - processed_data/X_val_robust.npy
  - processed_data/X_test_robust.npy

✓ Saved feature names:
  - processed_data/feature_names.csv (102 features)

✓ Saved scalers and encoders (pickle):
  - processed_data/scaler_standard.pkl
  - processed_data/scaler_minmax.pkl
  - processed_data/scaler_ro

In [12]:
"""
FINAL SUMMARY
=============
Complete overview of the data cleaning pipeline
"""
print("\n" + "=" * 80)
print("🎉 DATA CLEANING PIPELINE COMPLETE!")
print("=" * 80)

print("\n📊 PIPELINE SUMMARY:")
print(f"  1. ✓ Loaded listings_details.csv: {listings_df.shape}")
print(f"  2. ✓ Loaded calendar.csv: {calendar_df.shape}")
print(f"  3. ✓ Aggregated calendar features by listing_id")
print(f"  4. ✓ Merged with listings: {df.shape}")
print(f"  5. ✓ Split data (60/20/20):")
print(f"       - Train: {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"       - Val:   {len(X_val):,} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"       - Test:  {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")
print(f"  6. ✓ Applied preprocessing function to all splits")
print(f"  7. ✓ Applied 3 different scalers")
print(f"  8. ✓ Saved all processed data")

print("\n📈 FEATURE ENGINEERING:")
print(f"  Original features: {X_train.shape[1]}")
print(f"  Final features: {len(feature_names)}")
print(f"  Calendar features: {len([f for f in feature_names if 'calendar' in f.lower() or 'availability' in f.lower()])}")

calendar_feats = [f for f in feature_names if 'calendar' in f.lower() or 'availability' in f.lower()]
if calendar_feats:
    print(f"\n  Calendar-derived features:")
    for feat in calendar_feats:
        print(f"    • {feat}")

print("\n✅ DATA QUALITY CHECKS:")
print(f"  ✓ No data leakage (all transformations fit on train only)")
print(f"  ✓ No NaN values in scaled data")
print(f"  ✓ Proper train/val/test isolation maintained")
print(f"  ✓ All splits have same features: {X_train_clean.shape[1]} columns")

print("\n📁 OUTPUT FILES:")
print(f"  Directory: {output_dir}/")
print(f"  • Unscaled data (CSV): train_unscaled.csv, val_unscaled.csv, test_unscaled.csv")
print(f"  • Scaled data (NumPy):")
print(f"    - StandardScaler: X_train_standard.npy, X_val_standard.npy, X_test_standard.npy")
print(f"    - MinMaxScaler: X_train_minmax.npy, X_val_minmax.npy, X_test_minmax.npy")
print(f"    - RobustScaler: X_train_robust.npy, X_val_robust.npy, X_test_robust.npy")
print(f"  • Target arrays: y_train.npy, y_val.npy, y_test.npy")
print(f"  • Metadata: feature_names.csv, encoders.pkl, scaler_*.pkl")

print("\n🚀 READY FOR MODELING!")
print("  You can now load the processed data and train your models:")
print("  ")
print("  # Load unscaled data")
print("  train_df = pd.read_csv('processed_data/train_unscaled.csv')")
print("  ")
print("  # OR load scaled data")
print("  X_train = np.load('processed_data/X_train_standard.npy')")
print("  y_train = np.load('processed_data/y_train.npy')")
print("  ")
print("=" * 80)



🎉 DATA CLEANING PIPELINE COMPLETE!

📊 PIPELINE SUMMARY:
  1. ✓ Loaded listings_details.csv: (20030, 96)
  2. ✓ Loaded calendar.csv: (7310950, 6)
  3. ✓ Aggregated calendar features by listing_id
  4. ✓ Merged with listings: (20028, 102)
  5. ✓ Split data (60/20/20):
       - Train: 12,016 samples (60.0%)
       - Val:   4,006 samples (20.0%)
       - Test:  4,006 samples (20.0%)
  6. ✓ Applied preprocessing function to all splits
  7. ✓ Applied 3 different scalers
  8. ✓ Saved all processed data

📈 FEATURE ENGINEERING:
  Original features: 101
  Final features: 102
  Calendar features: 11

  Calendar-derived features:
    • has_availability
    • availability_30
    • availability_60
    • availability_90
    • availability_365
    • avg_calendar_price
    • min_calendar_price
    • max_calendar_price
    • availability_rate
    • calendar_days_count
    • calendar_available_days

✅ DATA QUALITY CHECKS:
  ✓ No data leakage (all transformations fit on train only)
  ✓ No NaN values in s